In [1]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.preprocessing.text import Tokenizer
from sklearn.model_selection import train_test_split

ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
!pip install konlpy

     |████████████████████████████████| 19.4MB 1.2MB/s 
     |████████████████████████████████| 3.8MB 48.8MB/s 
     |████████████████████████████████| 92kB 12.6MB/s 
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3
  Found existing installation: tweepy 3.6.0
    Uninstalling tweepy-3.6.0:
      Successfully uninstalled tweepy-3.6.0


In [ ]:
########## 한글 전처리부분입니다 ########

import pandas as pd
import numpy as np
from konlpy.tag import Okt
from sklearn.model_selection import train_test_split

naver_data = pd.read_table("ratings_test.txt", encoding='utf-8')
# 단어 아니면 삭제
print(naver_data)
naver_data['document'] = naver_data['document'].str.replace("[^\w]", " ")
# 혹시 공백이 있으면 null array로
naver_data['document'] = naver_data['document'].replace('', np.nan)
naver_data['label'] = naver_data['label'].replace('', np.nan)
# null array 모두 제거 (공백인 열 모두 제거)
naver_data = naver_data.dropna(how='any')

print("# preproecssing done")

# test/train 스플릿하고
review_train, review_test, y_train, y_test = train_test_split(naver_data['document'], naver_data['label'], test_size=0.05, shuffle=False, random_state=1004)

print('# split done')

# stopwords 지정
stopwords = ['를', '을', '이', '가', '으로', '로', '에', '에서']

# 토큰화 진행
X_train = []
for stc in review_train:
    token = []
    words = Okt().morphs(stc, stem=True)
    for word in words:
        if word not in stopwords:
            token.append(word)
    X_train.append(token)

X_test = []
for stc in review_test:
    token = []
    words = Okt().morphs(stc, stem=True)
    for word in words:
        if word not in stopwords:
            token.append(word)
    X_test.append(token)

print('# tokenization done')

# X_train 단어들을 토대로 정수 인덱스 설정, 전체 단어 갯수 설정
# 유의미한 단어? 빈도수가 1~2개인 단어 버려도 큰 영향을 끼치지 않을것 -> count함수써서 빈도수 낮은 것들을 버리고, 남은 단어의 갯수들
tokenizer = Tokenizer(20000)
tokenizer.fit_on_texts(X_train)

# 설정된 정수 인덱스를 토대로 변환
X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

print('# int_encoding done')

            id                                           document  label
0      6270596                                                굳 ㅋ      1
1      9274899                               GDNTOPCLASSINTHECLUB      0
2      8544678             뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아      0
3      6825595                   지루하지는 않은데 완전 막장임... 돈주고 보기에는....      0
4      6723715  3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??      0
...        ...                                                ...    ...
49995  4608761          오랜만에 평점 로긴했네ㅋㅋ 킹왕짱 쌈뽕한 영화를 만났습니다 강렬하게 육쾌함      1
49996  5308387       의지 박약들이나 하는거다 탈영은 일단 주인공 김대희 닮았고 이등병 찐따 OOOO      0
49997  9072549                 그림도 좋고 완성도도 높았지만... 보는 내내 불안하게 만든다      0
49998  5802125     절대 봐서는 안 될 영화.. 재미도 없고 기분만 잡치고.. 한 세트장에서 다 해먹네      0
49999  6070594                                         마무리는 또 왜이래      0

[50000 rows x 3 columns]
# preproecssing done
# split done
# tokenization done
# int_encoding done


In [ ]:
# 문장 길이를 맞춰준다
# 임의로 맞추는게 아니고, 데이터셋을 보면서 최대 문장의 길이가 얼마인지 확인하시고 거기에 맞춰서
# imdb일때는 500, 네이버일때는 50으로 맞췄었습니다!
max_len = 50
X_train = pad_sequences(X_train, maxlen=max_len)
X_test = pad_sequences(X_test, maxlen=max_len)

In [ ]:
# 레이어들을 쌓을 모델을 생성
model = Sequential()
# 단어를 임베딩하는데, 5000개 (imdb) 혹은 20000개 (네이버) 의 단어를 120차원으로 내보내겠다
# 1인자 = 내가 넣을 단어의 갯수 (총 인덱스의 갯수), 2인자 = 출력할 차원 (덴스 벡터의 차원), 3인자 = 문장의 길이
model.add(Embedding(20000, 120))
# RNN - simpleRNN / LSTM
model.add(LSTM(120))
# 긍정/부정을 판단하니까 이진 분류 -> sigmoid 함수 사용
model.add(Dense(1, activation='sigmoid'))

In [ ]:
# 혹시 5회 이상 검증데이터 loss가 증가하면, 과적합될 수 있으므로 학습을 조기종료!
early_stop = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5)
# 훈련을 거듭하면서, 가장 검증데이터 정확도가 높았던 순간을 체크포인트로 저장
model_check = ModelCheckpoint('the_best.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)

In [ ]:
# 긍정/부정을 판단하니까 손실함수는 이진 교차 엔트로피, 최적화는 adam, 평가 기준은 acc (출력할때 뜬다)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=1, batch_size=64, callbacks=[early_stop, model_check])

742/743 [============================>.] - ETA: 0s - loss: 0.4185 - acc: 0.8061
Epoch 00001: val_acc improved from -inf to 0.82560, saving model to the_best.h5
743/743 [==============================] - 22s 29ms/step - loss: 0.4185 - acc: 0.8061 - val_loss: 0.3891 - val_acc: 0.8256


In [ ]:
# 정확도 측정
# 출력하면 [loss, acc]
print(model.evaluate(X_test, y_test))

79/79 [==============================] - 0s 4ms/step - loss: 0.3891 - acc: 0.8256
[0.38906165957450867, 0.8256000280380249]


In [ ]:
model.save('webtoon_contents_model.h5')

In [ ]:
import re
# data['content'] = data['content'].str.replace("[^\w]", " ")
# # 혹시 공백이 있으면 null array로
# data['content'] = data['content'].replace('', np.nan)
# # null array 모두 제거 (공백인 열 모두 제거)
# data = data.dropna(how='any')
# test_data= ['계속 스토리 전개 안하고 과거 회상만 한다고 하는데. 그럼 대체 무슨 스토리 진행을 바라는거임? 목포에 도착하면 스토리는 거의 막바지일텐데 가는 동안 택시 안에서 보여줄 수 있는 스토리는 둘의 대화 말고는 없음.. 그리고 소설 ‘운수 좋은 날’ 에서 결말은 인력거꾼인 주인공이 그날 따라 수입이 좋고 운이 좋은 일이 많이 생겨 아픈 아내를 위해 설렁탕을 사갔으나 결국 아내는 죽어있더라 라는 결말임. 이미 저 택시 아저씨 가족의 죽음을 예상한 댓글들은 종종 보였고, 저 살인마가 가장 처음 저지른 살인부터 시간 순서대로 죽인 사람들을 회상과 대화를 통해 보여주고 마지막으로 기사에게 말해줄 내용은 택시 아저씨 가족을 죽인 이야기를 들려줄 것 같음.. 이 작품은 애초에 택시 안에서 목포까지 가는 과정을 보여주는 작품이고 그러면 대화와 회상이 작품의 주를 이루는게 당연한건데 대체 뭐가 불만들인지 잘 모르겠음',
#             '저정도 머리라면 지금 순간도 다 살인마의 계획일 수도 있겠군요. 이전부터 트렁크에 실려 있는 시신이 아내고 잘라온 손가락도 아내일거라는 추측이 있었는데, 이번 화로 그 추측이 맞다는 확신이 듭니다. 택시기사 또한 이전의 사람들처럼 철저하게 조사된 사람이고, 아내를 먼저 죽인 뒤 그가 택시를 통해 아내의 시신을 유기하는 것을 도왔다는 그림을 설계해서 모든 걸 알게 된 후의 택시기사의 반응을 기대하는 모습이 아닐까 생각이 드네요. 정신적 살해의 쾌감을 느낀 뒤라는 설정이 나온 만큼 이런 생각도 듭니다.',
#             '행님. 과거편이니 만큼 세세한 사건의 장황한 묘사보다 하나 하나 시원하게 떡밥 풀어가면서 휙휙 지나갔으면 하는데 그게 안되니까 원성을 사는겁니다. 덕형은 누구고, 피바다는 어떻게 죽었고, 광님이 곁에 슈빌은 우째 변했고, 구상룡 와꾸나 함 보고싶고, 이게 중헌거지 계속 그런건 안보여주고 새로운 인물 설정 스토리가 추가되니 지루하고 싸움에 관심이 없어지는 겁니다. 결국 우리가 궁금한건 저승세계고 그게 과거랑 어째 연결되는지 궁금한겁니다. 전개 다이어트좀 필요해보입니다.',
#             '아니 ㅋㅋㅋㅋ 유명한 연예인 집에 맨날 편지 썼다고 연예인이 바로 밥먹으러 집에 들어오라고 말하냐 그것도 2주만에 ㅋㅋ 말이 너무 안됨,,']
import csv
r=open('1스토리74전.csv', encoding='utf-8')
data=csv.reader(r)
a = []
for rr in data :
  

  test_data = rr
  test_data = [re.sub("[^\w]", " ",x) for x in test_data]
  # stopwords 지정
  stopwords = ['를', '을', '이', '가', '으로', '로', '에', '에서']
  x_test=[]
  stc_len = []
  for stc in test_data:
      
    token = []
    words = Okt().morphs(stc, stem=True)
    
    for word in words:
      if word not in stopwords:
          token.append(word)
    x_test.append(token)


  # 설정된 정수 인덱스를 토대로 변환
  x_test = tokenizer.texts_to_sequences(x_test)

  max_len = 50
  x_test = pad_sequences(x_test, maxlen=max_len)
  a.append(x_test)

# print(a[0])

Error: ignored

In [ ]:
# X 값은 전처리, 토큰화, 정수인코딩이 된 상태의 문장이어야
b = []

for i in a:
  predic=model.predict(i)
  # print(sum(predic)/100)
  b.append(predic.mean())
  

print(b)
print(type(b))
 
# for idx,len in enumerate(stc_len):
#   pre_sc = predic[idx]
#   print(pre[-len:])


[0.40619838, 0.409797, 0.41492286, 0.28784797, 0.31623304, 0.43101987, 0.49925303, 0.38592392, 0.41077664, 0.3377552, 0.4475031, 0.3313947, 0.46537098, 0.28870207, 0.3831992, 0.36070094, 0.4163593, 0.46140075, 0.33674204, 0.33526447, 0.36174327, 0.4324261, 0.42570847, 0.38322014, 0.42187434, 0.3935379, 0.48077136, 0.38047287, 0.48134923, 0.38897505, 0.46627152, 0.33963084, 0.44762257, 0.44848326, 0.46516943, 0.41340518, 0.48467723, 0.48713002, 0.50029725, 0.3811201, 0.37717932, 0.49644864, 0.40094468, 0.3850133, 0.4555124, 0.5348732, 0.39781675, 0.4030965, 0.393495, 0.4430431, 0.5114241, 0.54981124, 0.3844889, 0.4216792, 0.33020014, 0.41621044, 0.4107599, 0.47808638, 0.6330894, 0.44926196, 0.37399617, 0.5047824, 0.41720122, 0.354586, 0.42197305, 0.46271417, 0.39303, 0.36520746, 0.42120287, 0.34081152, 0.41944954, 0.45614308, 0.390689]
<class 'list'>
